# QuakeMigrate example - Icequake detection in Iceland

## Overview

This notebook shows how to run QuakeMigrate for icequake detection, using a 2 minute window of continuous seismic data from Hudson et al. (2019). Please refer to this paper for details and justification of the settings used.

Here, we detail how to:
1. Create travel-time lookup tables for the example seismometer network
2. Run the detect stage to coalesce energy through time
3. Run the trigger stage to determine events above a threshold value
4. Run the locate stage to refine the earthquake location

In [ ]:
import pandas as pd
from obspy.core import AttribDict
from pyproj import Proj

from quakemigrate.io import Archive
from quakemigrate.io import read_stations
from quakemigrate.lut import compute, LUT
from quakemigrate.signal import QuakeScan, Trigger
from quakemigrate.signal.onsets import STALTAOnset
from quakemigrate.signal.pickers import GaussianPicker

%matplotlib inline

In [ ]:
# --- i/o paths ---
station_file = "./inputs/iceland_stations.txt"
data_in = "./inputs/mSEED"
lut_out = "./outputs/lut/icequake.LUT"
run_path = "./outputs/runs"
run_name = "icequake_example"

## Create travel-time lookup tables (LUT)

In [ ]:
# --- Read in the station information file ---
stations = read_stations(station_file)

# --- Define the input and grid projections ---
gproj = Proj(proj="lcc", units="km", lon_0=-17.224, lat_0=64.328, lat_1=64.32,
             lat_2=64.335, datum="WGS84", ellps="WGS84", no_defs=True)
cproj = Proj(proj="longlat", datum="WGS84", ellps="WGS84", no_defs=True)

# --- Define the grid specifications ---
# The ObsPy AttribDict behaves like a Python dict, but with '.'-style access.
grid_spec = AttribDict()
grid_spec.ll_corner = [-17.24363934275664, 64.31947715407385, -1.390]
grid_spec.ur_corner = [-17.204348515198255, 64.3365202025144, 1.390]
grid_spec.node_spacing = [0.1, 0.1, 0.02]
grid_spec.grid_proj = gproj
grid_spec.coord_proj = cproj

# --- Homogeneous LUT generation ---
lut = compute_traveltimes(grid_spec, stations, method="homogeneous", vp=3.630,
                          vs=1.833, log=True, save_file=lut_out)

## Coalesce the seismic energy through time

In [ ]:
# --- Read in station file ---
stations = read_stations(station_file)

# --- Create new Archive and set path structure ---
archive = Archive(archive_path=data_in, stations=stations,
                  archive_format="YEAR/JD/*_STATION_*")

# --- Create new Onset ---
onset = STALTAOnset(position="classic")
onset.p_bp_filter = [10, 125, 4]
onset.s_bp_filter = [10, 125, 4]
onset.p_onset_win = [0.01, 0.25]
onset.s_onset_win = [0.05, 0.5]

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, run_path=run_path,
                 run_name=run_name, log=True)

# --- Set detect parameters ---
scan.sampling_rate = 500
scan.timestep = 0.75
scan.threads = 12

# --- Set time period over which to run detect ---
starttime = "2014-06-29T18:41:55.0"
endtime = "2014-06-29T18:42:20.0"

In [ ]:
# --- Run detect ---
scan.detect(starttime, endtime)

## Run the trigger stage, to detect and output individual icequakes

nb: We can use the same QuakeScan object here because we are not using a different decimation. If running trigger and locate on grids with different levels of decimation, a new QuakeScan object should be created.

In [ ]:
# --- Create new Trigger ---
trig = Trigger(lut, run_path=run_path, run_name=run_name)

# --- Set trigger parameters ---
trig.marginal_window = 1.
trig.min_event_interval = 6.
trig.normalise_coalescence = True

# --- Static threshold ---
trig.threshold_method = "static"
trig.static_threshold = 1.8

# --- Run trigger ---
trig.trigger(starttime, endtime, interactive_plot=False)

## Run the locate stage, to relocate triggered events on a less decimated grid

In [ ]:
# --- Create new PhasePicker ---
picker = GaussianPicker(onset=onset)
picker.marginal_window = 2.75
picker.plot_picks = True

# --- Create new QuakeScan ---
scan = QuakeScan(archive, lut, onset=onset, picker=picker,
                 run_path=run_path, run_name=run_name, log=True)

# --- Set locate parameters ---
scan.marginal_window = 1.
scan.threads = 12
scan.sampling_rate = 500

# --- Toggle plotting options ---
scan.plot_event_summary = True
scan.plot_event_video = False

# --- Toggle writing of waveforms ---
scan.write_cut_waveforms = False

In [ ]:
# --- Run locate ---
scan.locate(starttime=starttime, endtime=endtime)

## Some of the key outputs

In [ ]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = "./outputs/runs/icequake_example/locate/events/20140629184210348.event"
event_df = pd.read_csv(icequake_event_fname)

event_df

In [ ]:
# Show the .picks file, containing station time picks:
icequake_pick_fname = "outputs/runs/icequake_example/locate/picks/20140629184210348.picks"
pick_df = pd.read_csv(icequake_pick_fname)

pick_df

In [ ]:
# Show the coalescence pdf file, containing event origin time and location:
icequake_coal_image_fname = "outputs/runs/icequake_example/locate/summaries/icequake_example_20140629184210348_EventSummary.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_coal_image_fname, width=800, height=400) # Plot pdf

References:

Hudson, T.S., Smith, J., Brisbourne, A.M., and White R.S. (2019). Automated detection of basal icequakes and discrimination from surface crevassing. Annals of Glaciology, 79